# Hyper Parameters

In [1]:
notebookName = "BaseModel"
nepochs = 10
batch_size = 4
learning_rate = 0.001

SEED = 20180724
target_layer = ['road_block', 'walkway', 'road_divider', 'traffic_light']
class_names = ['None'] + target_layer
MAX_OBJECTS = 1
MAX_POINTS = 30
MAX_POINT_CLOUDS = 1200

In [2]:
LIDAR_PC_SHAPE = [4, MAX_POINT_CLOUDS] # [x,y,z,intensity] x num_of_points
MAP_OBJECT_SHAPE = [MAX_OBJECTS, MAX_POINTS, 2] # num_of_objects x num_of_points x [x, y]
MAP_LAYER_SHAPE = [len(class_names), MAX_OBJECTS] # num_of_class x num_of_objects
PATCH_SIZE = [-1, -1]

# Base Setting

In [3]:
import os
from pathlib import Path
import time

from nuscenes.nuscenes import NuScenes
from utils.custom_lidar_api import CustomLidarApi
from utils.custom_map_api_expansion import CustomNuScenesMap

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
from torch.nn import Module
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import TensorDataset, DataLoader

import warnings
warnings.filterwarnings(action='ignore')

%matplotlib inline 

In [4]:
locations = ['singapore-onenorth', 'singapore-hollandvillage', 'singapore-queenstown', 'boston-seaport']
version = 'v1.0-trainval'
dataroot = 'E:/datasets/nuscenes'

In [5]:
torch.manual_seed(SEED)
np.random.seed(SEED)

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [7]:

PATH = Path(f"./models/{notebookName}")
if os.path.isdir(PATH):
    dir_list = os.listdir(PATH)
    num_files = 0
    while True:
        if os.path.isfile(str(PATH / f"{num_files}")):
            print(num_files)
            num_files += 1
        else:
            break
else:
    os.mkdir(PATH)
    num_files = 0
num_files

0

In [8]:
class_dict = dict()

for i, name in enumerate(target_layer):
    class_dict[name] = i+1
class_array = np.eye(len(class_names))

# Load Nusc, Map Api, and Ldr Api

In [9]:
nusc = NuScenes(version=version, dataroot=dataroot, verbose=True)

Loading NuScenes tables for version v1.0-trainval...
Loading nuScenes-lidarseg...
32 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
34149 lidarseg,
Done loading in 59.870 seconds.
Reverse indexing ...
Done reverse indexing in 8.9 seconds.


In [10]:
map_api = dict([])
for location in locations:
    map_api[location] = CustomNuScenesMap(dataroot = dataroot, map_name= location, target_layer_names=target_layer, max_objs=MAX_OBJECTS, max_points=MAX_POINTS)

In [11]:
ldr_api = CustomLidarApi(nusc)

In [12]:
# get all sample token
sample_tokens = []
for scene in nusc.scene:
    token = scene['first_sample_token']
    while token != scene['last_sample_token']:
        sample_tokens.append(token)
        sample = nusc.get('sample', token)
        token = sample['next']
print(len(sample_tokens))

33299


In [13]:
def train_val_split(sample_tokens, ratio = 0.1, shuffle = True):
    index = np.array(range(len(sample_tokens)))
    index = np.random.choice(index.shape[0], index.shape[0], replace = False)
    
    valid_num = int(index.shape[0] * ratio)
    
    valid_tokens = sample_tokens[:valid_num]
    train_tokens = sample_tokens[valid_num:]
    
    return train_tokens, valid_tokens

train_tokens, valid_tokens = train_val_split(sample_tokens)

print(len(sample_tokens))
print(len(train_tokens))
print(len(valid_tokens))

33299
29970
3329


# Customized Dataset

In [14]:
# LIDAR_PC_SHAPE = [4, MAX_POINT_CLOUDS] # [x,y,z,intensity] x num_of_points
# MAP_OBJECT_SHAPE = [MAX_OBJECTS, MAX_POINTS, 2] # num_of_objects x num_of_points x [x, y]
# MAP_LAYER_SHAPE = [len(class_names), MAX_OBJECTS] # num_of_class x num_of_objects
class NusceneDataset(Dataset):
    def __init__(self, tokens, nusc, map_api, ldr_api, train = True):
        self.tokens = tokens
        self.nusc = nusc
        self.map_api = map_api
        self.ldr_api = ldr_api
        self.train = train
        
        self.length = len(tokens)
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        token = self.tokens[idx]
        
        sample = self.nusc.get('sample', token)
        scene = self.nusc.get('scene', sample['scene_token'])
        log_meta = self.nusc.get('log', scene['log_token'])
        
        location = log_meta['location']
        sample_data = self.nusc.get('sample_data', sample['data']['LIDAR_TOP'])
        
        pc = self.ldr_api.get_lidar_from_keyframe(token, max_points = LIDAR_PC_SHAPE[1], car_coord = True)
        ego = self.ldr_api.get_egopose_from_keyframe(token)
        structures = self.map_api[log_meta['location']].get_closest_structures(ego, \
                                                                               patch = PATCH_SIZE,\
                                                                               global_coord=False, \
                                                                               mode = 'intersect'\
                                                                              )
        if(len(structures) == 0):
            print(log_meta['location'])
            print(PATCH_SIZE)
            print(token)
            print(ego)
            print(len(structures))
            print(structures)
            print(self.map_api[log_meta['location']].structures[:3])
            print(self.map_api[log_meta['location']])
        
        X = torch.Tensor(pc.points)
        if self.train:
            classes, objects = self.get_label(structures)
        else:
            classes, objects = self.get_label(list())
        
        return X, classes, objects
    
    def get_label(self, structures):
        classes = list(map(lambda x: torch.Tensor(class_array[class_dict[x["layer"]], :]).reshape(MAP_LAYER_SHAPE[0], MAP_LAYER_SHAPE[1]), structures))
        classes = torch.cat(classes, axis = 1)
        
        objects = list(map(lambda x: torch.Tensor(x['nodes'].reshape(MAP_OBJECT_SHAPE[0], MAP_OBJECT_SHAPE[1], MAP_OBJECT_SHAPE[2])), structures))
        objects = torch.cat(objects, axis = 0)
        
        return classes, objects
        

train_dataset = NusceneDataset(tokens = train_tokens, nusc = nusc, map_api = map_api, ldr_api = ldr_api)
valid_dataset = NusceneDataset(tokens = valid_tokens, nusc = nusc, map_api = map_api, ldr_api = ldr_api)

train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
valid_loader = DataLoader(valid_dataset, batch_size = batch_size, shuffle = False)

In [15]:
# def get_label(structures):
#     classes = list(map(lambda x: torch.Tensor(class_array[class_dict[x["class"]], :]).reshape(MAP_LAYER_SHAPE[0], MAP_LAYER_SHAPE[1]), structures))
#     classes = torch.cat(classes, axis = 1)

#     objects = list(map(lambda x: torch.Tensor(x['nodes'].reshape(MAP_OBJECT_SHAPE[0], MAP_OBJECT_SHAPE[1], MAP_OBJECT_SHAPE[2])), structures))
#     objects = torch.cat(objects, axis = 0)
#     return classes, objects

# %load_ext line_profiler

# token = "54ea5283e6d643b4b54fe489bd373c16"

# sample = nusc.get('sample', token)
# scene = nusc.get('scene', sample['scene_token'])
# log_meta = nusc.get('log', scene['log_token'])
# location = log_meta['location']
# sample_data = nusc.get('sample_data', sample['data']['LIDAR_TOP'])
# pc = ldr_api.get_lidar_from_keyframe(token, max_points = LIDAR_PC_SHAPE[1], car_coord = True)
# ego = ldr_api.get_egopose_from_keyframe(token)
# mode = 'intersect'
# %lprun -f  map_api[location].get_closest_structures map_api[location].get_closest_structures(ego, patch = [200, 200],mode = mode,global_coord=False)


In [16]:
for X, c, y in tqdm(train_loader):
    pass

  0%|          | 0/7493 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Model

In [17]:
# LIDAR_PC_SHAPE = [4, MAX_POINT_CLOUDS] # [x,y,z,intensity] x num_of_points
# MAP_OBJECT_SHAPE = [MAX_OBJECTS, MAX_POINTS, 2] # num_of_objects x num_of_points x [x, y]
# MAP_LAYER_SHAPE = [len(class_names), MAX_OBJECTS] # num_of_class x num_of_objects
class BaseModel(torch.nn.Module):
    def __init__(self, input_shape = [4, 3500], class_shape = [4, 1], object_shape = [1, 30, 2]):
        super().__init__()
        self.input_shape =input_shape
        self.object_shape = object_shape
        self.class_shape = class_shape
        
        self.conv1 = nn.Conv1d(input_shape[0], 64, kernel_size=1)
        self.batch1 = nn.BatchNorm1d(64)
        
        self.conv2 = nn.Conv1d(64, 64, kernel_size=1)
        self.batch2 = nn.BatchNorm1d(64)
        
        self.fc1 = nn.Linear(64 * input_shape[1], 1024)
        self.fc2 = nn.Linear(1024, 512)
        
        self.fc_obj = nn.Linear(512, np.prod(object_shape))
        self.fc_cls = nn.Linear(512, np.prod(class_shape))
        
    def forward(self, x):
        input_shape = self.input_shape
        object_shape = self.object_shape
        class_shape = self.class_shape
        
        x = self.conv1(x)
        x = F.relu(x)
        x = self.batch1(x)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.batch2(x)
        
        x = x.reshape(-1, input_shape[1] * 64)
        
        x = self.fc1(x)
        x = F.relu(x)
        
        x = self.fc2(x)
        x = F.relu(x)
        
        obj_ = self.fc_obj(x)
        cls_ = self.fc_cls(x)
        
        obj_ = obj_.reshape(-1, object_shape[0], object_shape[1], object_shape[2])
        cls_ = cls_.reshape(-1, class_shape[0], class_shape[1])
        return cls_, obj_

# Loss

In [18]:
class BaseLoss(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.pose_loss = nn.MSELoss()
        self.class_loss = nn.CrossEntropyLoss()
        pass
    def forward(self, c_hat, y_hat, c, y):
        
        loss = self.pose_loss(y, y_hat) + self.class_loss(c_hat, c.argmax(axis = 1))
        return loss

# Model Compile

In [19]:
model = BaseModel(LIDAR_PC_SHAPE, MAP_LAYER_SHAPE, MAP_OBJECT_SHAPE)
model.to(device)

loss_func= BaseLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001)

# Train Module

In [20]:
def train(epoch, progress_log):
    model.train()  # 신경망을 학습 모드로 전환

    # 데이터로더에서 미니배치를 하나씩 꺼내 학습을 수행
    mean_loss = 0
    data_num = 0
    
    for X, c, y in progress_log:
        
        X = X.to(device)
        c = c.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()  # 경사를 0으로 초기화
        c_hat, y_hat = model(X)  # 데이터를 입력하고 출력을 계산
        loss = loss_func(c_hat, y_hat, c, y)  # 출력과 훈련 데이터 정답 간의 오차를 계산
        
        loss.backward()  # 오차를 역전파 계산
        optimizer.step()  # 역전파 계산한 값으로 가중치를 수정
        
        mean_loss += loss
        data_num += X.shape[0]
        
    mean_loss /= data_num
    
    return mean_loss

# Valid Module

In [21]:
def valid(epoch, progress_log):
    model.eval()  # 신경망을 학습 모드로 전환

    # 데이터로더에서 미니배치를 하나씩 꺼내 학습을 수행
    mean_loss = 0
    data_num = 0
    
    with torch.no_grad():
        for X, c, y in progress_log:

            X = X.to(device)
            c = c.to(device)
            y = y.to(device)

            c_hat, y_hat = model(X)  # 데이터를 입력하고 출력을 계산
            loss = loss_func(c_hat, y_hat, c, y)  # 출력과 훈련 데이터 정답 간의 오차를 계산

            mean_loss += loss
            data_num += X.shape[0]
        
    mean_loss /= data_num
    
    return mean_loss

# Test Module

In [22]:
def test(epoch, progress_log):
    model.eval()  # 신경망을 학습 모드로 전환

    # 데이터로더에서 미니배치를 하나씩 꺼내 학습을 수행
    C_hat = []
    Y_hat = []
    
    with torch.no_grad():
        for X, _, _ in progress_log:

            X = X.to(device)

            c_hat, y_hat = model(X)  # 데이터를 입력하고 출력을 계산
            C_hat.append(c_hat)
            Y_hat.append(y_hat)
        
    C_hat = np.concatenate(C_hat)
    Y_hat = np.concatenate(Y_hat)
    
    return C_hat, Y_hat

# Fit

In [ ]:
train_loss_list = []
valid_loss_list = []

patience_count = 0
min_valid_loss = np.inf
checkpoint_name = ""

if not os.path.isdir(f"./models/{notebookName}/model-{num_files}_checkpoint/"):
    os.mkdir(f"./models/{notebookName}/model-{num_files}_checkpoint/")
    
prog_epoch = tqdm(range(0, nepochs), position = 0, desc = 'EPOCH')
for epoch in prog_epoch:
    print( "-------------------------------------------------------")
    print(f"|EPOCH: {epoch+1}/{nepochs}")
    prog_train = tqdm(train_loader, desc = 'TRAIN', leave = False)
    prog_valid = tqdm(valid_loader, desc = 'VALID', leave = False)

    train_loss = train(epoch, prog_train)
    valid_loss = valid(prog_valid)
    
    if valid_loss < min_valid_loss:
        print(f"|{epoch+1}-th model is checked!, *model-{epoch}-{valid_loss}.pth*")
        min_valid_loss= valid_loss
        checkpoint_name = f"./models/{notebookName}/model-{num_files}_checkpoint/model-{epoch}-{valid_loss}.pth"
        torch.save(model.state_dict(), checkpoint_name)
    else:
        patience_count+=1
        if(patience_count > max_patience_count):
            break
    
    train_loss_list.append(train_loss)
    valid_loss_list.append(valid_loss)
    
    print(f"|TRAIN: loss={train_loss:.6f}|")
    print(f"|VALID: loss={valid_loss:.6f}|")


history = dict()
history['train_loss'] = train_loss_list
history['valid_loss'] = valid_loss_list

EPOCH:   0%|          | 0/10 [00:00<?, ?it/s]

-------------------------------------------------------
|EPOCH: 1/10


TRAIN:   0%|          | 0/7493 [00:00<?, ?it/s]

VALID:   0%|          | 0/833 [00:00<?, ?it/s]

In [ ]:
plt.figure(figsize = (16,6))
plt.subplot(2,1,1)
plt.plot(history['train_loss'], label = 'train')
plt.plot(history['valid_loss'], label = 'valid')
plt.ylabel('loss')

plt.subplot(2,1,2)
plt.plot(history['train_score'], label = 'train')
plt.plot(history['valid_score'], label = 'valid')
plt.ylabel('gpsloss')